In [42]:
# Imports
from pandas import json_normalize
import requests
import json
import os
import re
import time
import fnmatch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io import sql

# Download NLTK resources if not already downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/salomekoller/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
# Read file into dataframe
df_tweets = pd.read_csv('../01_Kontext/10_datasource/interaction_with_date_and_time.csv', sep=',', encoding='utf-8')
print(df_tweets.shape)
df_tweets.head(5)

(64345, 7)


,interaction_id,tweet_id,senderuser_id,receiveruser_id,tweet_text,cyberbullying_type,interaction_timestamp
0,1304,1510,876,2009,@g56yu What is that?,not_cyberbullying,41:34.9
1,1998,31503,918,2436,@stockputout yes,other_cyberbullying,37:48.7
2,2533,27942,1409,3624,5/13/15,other_cyberbullying,50:46.1
3,2598,52129,1341,3290,@Blackamazon EXACTLY THIS.,none,04:51.4
4,3511,54626,831,2812,@Oddtankout http://t.co/K5mMJE8UME,none,11:11.2


In [44]:
df_tweets['cyberbullying_type'].unique()

array(['not_cyberbullying', 'other_cyberbullying', 'none', 'gender', nan,
       'sexism', 'age', 'ethnicity', 'religion', 'racism'], dtype=object)

In [45]:
# Change strings to uppercase 
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.upper()
print(df_tweets['tweet_text'].head(10), '\n')

0                  @G56YU WHAT IS THAT?
1                      @STOCKPUTOUT YES
2                               5/13/15
3            @BLACKAMAZON EXACTLY THIS.
4    @ODDTANKOUT HTTP://T.CO/K5MMJE8UME
5                                  ROFL
6              OH, YES HE IS A BAD GUY.
7                HTTP://T.CO/NXGTQSLCHG
8                             ¯\_(ツ)_/¯
9                        GAY RAPE JOKE?
Name: tweet_text, dtype: object 



In [46]:
# ChatGTP has created the pattern 
# Create a pattern which can be used to search the variable 'tweet_text'
patternHarass = '(INSULT)|(ABUSE)|(THREATEN)|(VICTIM)|(TORMENT)|(PERSECUTE)|(INTIMIDATE)|(AGGRESSION)|(RELENTLESS)|(BULLYING)|(HOSTILITY)|(SLANDER)|(HURTFUL)|(ATTACKS)|(OFFENSE)|(MALICIOUS)|(STALKER)|(TAUNT)|(ABUSIVE)|(TARGET)'
patternCyberstalk = '(STALK)|(PURSUE)|(OBSESS)|(WATCH)|(FOLLOW)|(INTRUDE)|(SPY)|(UNWANTED)|(MONITORING)|(SURVEILLANCE)|(TRACKING)|(INVASION)|(SHADOW)|(PRYING)|(PERSISTENCE)|(STALKING)|(SPYING)|(INTRUSIVE)|(PERVASION)|(PURSUIT)'
patternFlaming = '(PROVOCATION)|(INSULTING)|(AGGRESSIVE)|(HOSTILE)|(OFFEND)|(HEATED)|(ARGUMENTATIVE)|(OFFENSIVE)|(CONTENTIOUS)|(DISPUTE)|(ANTAGONISTIC)|(FIERY)|(INCENDIARY)|(HARSH)|(COMBATIVE)|(CLASHING)|(ARGUMENT)|(PROVOCATIVE)|(SPARRING)|(CONTROVERSIAL)'
patternOuting = '(EXPOSURE)|(REVEAL)|(DISCLOSURE)|(BETRAYAL)|(UNMASKING)|(UNVEILING)|(LEAKAGE)|(PUBLICATION)|(BETRAYED)|(UNCOVERING)|(EXPOSED)|(CONFESSION)|(REVEALING)|(LEAKED)|(PRIVACY)|(PRIVATE)|(DATA)|(DETAILS)|(IDENTIFY)|(PUBLISH)'
patternDoxing = '(DOX)|(INFORMATION)|(REVEAL)|(DATA)|(PERSONAL)|(IDENTITY)|(PRIVACY)|(EXPOSED)|(PUBLISH)|(UNMASKING)|(LEAKED)|(PROFILE)|(ADDRESS)|(DETAILS)|(IDENTIFY)|(PUBLISH)|(REVEALING)|(EXPOSE)|(DATA)|(UNVEILING)'
patternExclusion = '(ISOLATE)|(OSTRACIZE)|(EXCLUDE)|(ALIENATION)|(SHUN)|(REJECT)|(IGNORED)|(OUTSIDER)|(SOLITUDE)|(SEGREGATE)|(ABANDONMENT)|(NEGLECTED)|(ISOLATED)|(MARGINALIZED)|(UNWANTED)|(LONELINESS)|(REJECTION)|(ISOLATION)|(NEGLECT)|(NEGLECTING)'
patternImpersonation = '(FAKE)|(IDENTITY)|(DECEPTION)|(FRAUDULENT)|(PRETEND)|(IMITATE)|(COUNTERFEIT)|(MIMIC)|(PHONY)|(FRAUD)|(FORGERY)|(IMPOSTER)|(DECEIVER)|(FAKE PROFILE)|(FALSE IDENTITY)|(IDENTITY THEFT)|(FRAUDULENT ACCOUNT)|(MIMICKING)|(FORGED)|(COUNTERFEITING)'
patternTrolling = '(TROLL)|(PROVOCATION)|(AGITATION)|(DISRUPTION)|(ANNOYING)|(NUISANCE)|(IRRITATION)|(HARASSMENT)|(MISCHIEF)|(TEASING)|(INSTIGATION)|(OFFENSIVE)|(MOCKERY)|(RIDICULE)|(TAUNT)|(DISTURBANCE)|(BAITING)|(ANTAGONISTIC)|(CONTROVERSIAL)|(PRANKS)'
patternHateSpeech = '(HATE)|(BIGOTRY)|(DISCRIMINATION)|(PREJUDICE)|(STEREOTYPING)|(HOMOPHOBIA)|(MISOGYNY)|(RACIAL)|(SLURS)|(XENOPHOBIA)|(BIAS)|(OFFENSIVE)|(DEROGATORY)|(VILE)|(PREJUDICED)|(HATE-FILLED)|(INTOLERANCE)|(RACIST)|(SEXIST)|(HOMOPHOBIC)'
patternThreats = '(THREATEN)|(VIOLENCE)|(DANGER)|(MENACE)|(INTIMIDATE)|(HARM)|(PERIL)|(WARNING)|(COERCION)|(FEAR)|(HARBINGER)|(TERRORIZE)|(MENACING)|(ASSAULT)|(HOSTILITY)|(AGGRESSION)|(THREATENING)|(THREATENED)|(ABUSIVE)|(VIOLENT)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df_tweets['harass'] = df_tweets['tweet_text'].str.contains(pat = patternHarass).astype(float)
print(df_tweets['harass'].sum())
df_tweets['cyberstalk'] = df_tweets['tweet_text'].str.contains(pat = patternCyberstalk).astype(float)
print(df_tweets['cyberstalk'].sum())
df_tweets['flaming'] = df_tweets['tweet_text'].str.contains(pat = patternFlaming).astype(float)
print(df_tweets['flaming'].sum())
df_tweets['outing'] = df_tweets['tweet_text'].str.contains(pat = patternOuting).astype(float)
print(df_tweets['outing'].sum())
df_tweets['doxing'] = df_tweets['tweet_text'].str.contains(pat = patternDoxing).astype(float)
print(df_tweets['doxing'].sum())
df_tweets['exclusion'] = df_tweets['tweet_text'].str.contains(pat = patternExclusion).astype(float)
print(df_tweets['exclusion'].sum())
df_tweets['impersonation'] = df_tweets['tweet_text'].str.contains(pat = patternImpersonation).astype(float)
print(df_tweets['impersonation'].sum())
df_tweets['trolling'] = df_tweets['tweet_text'].str.contains(pat = patternTrolling).astype(float)
print(df_tweets['trolling'].sum())
df_tweets['hateSpeech'] = df_tweets['tweet_text'].str.contains(pat = patternHateSpeech).astype(float)
print(df_tweets['hateSpeech'].sum())
df_tweets['threats'] = df_tweets['tweet_text'].str.contains(pat = patternThreats).astype(float)
print(df_tweets['threats'].sum())

# Show values
df_tweets[['tweet_text','harass','cyberstalk','flaming','outing', 'doxing', 'exclusion', 'impersonation', 'trolling', 'hateSpeech', 'threats']]

/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_24133/403006207.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['harass'] = df_tweets['tweet_text'].str.contains(pat = patternHarass).astype(float)


3285.0


/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_24133/403006207.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['cyberstalk'] = df_tweets['tweet_text'].str.contains(pat = patternCyberstalk).astype(float)


2285.0


/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_24133/403006207.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['flaming'] = df_tweets['tweet_text'].str.contains(pat = patternFlaming).astype(float)


...

/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_24133/403006207.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['hateSpeech'] = df_tweets['tweet_text'].str.contains(pat = patternHateSpeech).astype(float)


6890.0


/var/folders/y8/blsng9yn0_jdwkc6k9fx0f240000gn/T/ipykernel_24133/403006207.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_tweets['threats'] = df_tweets['tweet_text'].str.contains(pat = patternThreats).astype(float)


1605.0


,tweet_text,harass,cyberstalk,flaming,outing,doxing,exclusion,impersonation,trolling,hateSpeech,threats
0,@G56YU WHAT IS THAT?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,@STOCKPUTOUT YES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5/13/15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,@BLACKAMAZON EXACTLY THIS.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,@ODDTANKOUT HTTP://T.CO/K5MMJE8UME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
64340,ONLY THING THAT HEATS MY HEAD MORE IS WHEN I S...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64341,FOUND OUT THE GIRL WHO SPREAD (TRUE) RUMOURS A...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
64342,"NIÑAS, REINAS DEL “BULLYING” EN ESCUELAS PRIVA...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64343,"@DIANH4 AS USUAL, YOU ARE A MUSLIM LIAR MAKING...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df_tweets['word_count'] = df_tweets['tweet_text'].apply(lambda x: len(str(x).split()))
df_tweets['char_count'] = df_tweets['tweet_text'].apply(lambda x: len(str(x)))
df_tweets[['tweet_text', 'word_count', 'char_count']].head(5)

,tweet_text,word_count,char_count
0,@G56YU WHAT IS THAT?,4,20
1,@STOCKPUTOUT YES,2,16
2,5/13/15,1,7
3,@BLACKAMAZON EXACTLY THIS.,3,26
4,@ODDTANKOUT HTTP://T.CO/K5MMJE8UME,2,34


In [48]:
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Custom functions to calculate text statistics
def average_word_length(tweet_text):
    if isinstance(tweet_text, str):  # Check if tweet_text is a string
        words = word_tokenize(tweet_text)
        return sum(len(word) for word in words) / len(words)
    else:
        return 0  # or any default value for non-string entries

def average_sentence_length(tweet_text):
    if isinstance(tweet_text, str):
        sentences = sent_tokenize(tweet_text)
        return len(word_tokenize(tweet_text)) / len(sentences)
    else:
        return 0

def count_punctuation(tweet_text):
    if isinstance(tweet_text, str):
        return sum(1 for char in tweet_text if char in string.punctuation)
    else:
        return 0

# Apply custom functions to calculate text statistics
df_tweets['avg_word_length'] = df_tweets['tweet_text'].apply(average_word_length)
df_tweets['avg_sentence_length'] = df_tweets['tweet_text'].apply(average_sentence_length)
df_tweets['punctuation_count'] = df_tweets['tweet_text'].apply(count_punctuation)

# Display the DataFrame with text statistics
df_tweets[['tweet_text', 'avg_word_length', 'avg_sentence_length', 'punctuation_count']].head(5)

,tweet_text,avg_word_length,avg_sentence_length,punctuation_count
0,@G56YU WHAT IS THAT?,2.833333,6.0,2
1,@STOCKPUTOUT YES,5.000000,3.0,1
2,5/13/15,7.000000,1.0,2
3,@BLACKAMAZON EXACTLY THIS.,4.800000,5.0,2
4,@ODDTANKOUT HTTP://T.CO/K5MMJE8UME,6.600000,5.0,6


In [49]:
zero_values_count = (df_tweets['avg_word_length'] == 0).sum()
print(f'Number of 0 values in avg_word_length: {zero_values_count}')


Number of 0 values in avg_word_length: 1


Check, on how many 0 Values have been created.

In [50]:
print(f'Number of entries with avg_word_length as 0: {(df_tweets["avg_word_length"] == 0).sum()}')
print(f'Number of entries with avg_sentence_length as 0: {(df_tweets["avg_sentence_length"] == 0).sum()}')
print(f'Number of entries with punctuation_count as 0: {(df_tweets["punctuation_count"] == 0).sum()}')


Number of entries with avg_word_length as 0: 1
Number of entries with avg_sentence_length as 0: 1
Number of entries with punctuation_count as 0: 3044


Looking into the ones that have 0 as a value:

In [51]:
print(df_tweets[df_tweets['avg_word_length'] == 0][['tweet_text', 'avg_word_length']])

   tweet_text  avg_word_length
13        NaN              0.0


In [52]:
print(df_tweets[df_tweets['avg_sentence_length'] == 0][['tweet_text', 'avg_sentence_length']])

   tweet_text  avg_sentence_length
13        NaN                  0.0


In [53]:
print(df_tweets[df_tweets['punctuation_count'] == 0][['tweet_text', 'punctuation_count']])

                                              tweet_text  punctuation_count
5                                                   ROFL                  0
13                                                   NaN                  0
27                        TOURE JUST HIGH SCHOOL BULLIES                  0
35                                               SOCIAIS                  0
40                      NAKED IS THE NEW LOOK NOWAH DAYS                  0
...                                                  ...                ...
64298                                   HEE HEE HEEEEEEE                  0
64311                    I WANNA HAVE A FEMINAZI NIGHT 😬                  0
64322  THE SPONGEBOB YTPS FROM 2007 SUCK THOUGH BECAU...                  0
64326  THE WEIRD ANIME KID AFTER THE SCHOOL BULLY DIS...                  0
64329                                           FEMINAZI                  0

[3044 rows x 2 columns]


The ones that have 0 values are explainable due to the values presented, like the punctuation count is 0 because there is no punctuation.

In [54]:
df_tweets.dtypes

interaction_id             int64
tweet_id                   int64
senderuser_id              int64
receiveruser_id            int64
tweet_text                object
cyberbullying_type        object
interaction_timestamp     object
harass                   float64
cyberstalk               float64
flaming                  float64
outing                   float64
doxing                   float64
exclusion                float64
impersonation            float64
trolling                 float64
hateSpeech               float64
threats                  float64
word_count                 int64
char_count                 int64
avg_word_length          float64
avg_sentence_length      float64
punctuation_count          int64
dtype: object

In [55]:
df_tweets.to_csv('dataset_tweets_featured.csv',
                      sep=",",
                      encoding='utf-8',
                      index=False)